# Packages

In [5]:
from numpy import *
from prody import *
from matplotlib.pyplot import *
from scipy.stats import hypergeom
import os

In [6]:
from Hinges_vs_Drugs import *

# Build ensembles

In [27]:
from collections import defaultdict

def mergeChains(currPDB, chain_results, original_chains):
    """
    Merges multi-chain Dali results by identifying PDBs that contain
    homologous chains for all original chains, ensuring distinct chain usage.
    Avoids duplicates and excludes self-chain matches (e.g., AA or BB).
    """
    pdb_map = defaultdict(lambda: defaultdict(set))

    for orig_chain, pdb_set in chain_results.items():
        for pdb in pdb_set:
            pdb_id, chain_id = pdb[:4], pdb[4:]
            pdb_map[pdb_id][orig_chain].add(chain_id)

    seen = set()
    final_results = [currPDB]  # Always include the reference input (PDB ID or file name)

    for pdb_id, chains_by_orig in pdb_map.items():
        # Step 1: check if this pdb_id is in ALL input chains
        if not all(oc in chains_by_orig for oc in original_chains):
            continue

        # Step 2: try to form a combination of different chains across original chains
        valid = True
        combo = []
        used_chains = set()

        for oc in original_chains:
            candidates = chains_by_orig[oc]
            # Pick first unused chain ID
            chosen = None
            for c in candidates:
                if c not in used_chains:
                    chosen = c
                    break
            if chosen is None:
                valid = False
                break
            combo.append(chosen)
            used_chains.add(chosen)

        if not valid:
            continue

        # Step 3: create final string and avoid duplicate combos like AB vs BA
        chain_str = ''.join(sorted(combo))
        tag = f"{pdb_id}{chain_str}"
        if tag not in seen:
            seen.add(tag)
            final_results.append(tag)

    if len(final_results) > 1:
        return final_results[1:]
    else:
        return []


def getEnsembleWithModes(currPDB, chains, mode="auto", length=None, rmsd=None, Z=10):
    """
    Returns a list of homologous structure IDs using Dali (with local .pdb or PDB ID).
    Supports both monomer and multimer chains.

    If a .pdb file is used, it is inserted as the first element of the returned list.

    :param currPDB: str - PDB ID or local .pdb file
    :param chains: str - single or multiple chain IDs (e.g., "A" or "AB")
    :param mode: "similar", "diverse", or "auto"
    :param length: float - Dali alignment cutoff
    :param rmsd: float - Dali RMSD filter
    :param Z: float - Dali Z-score filter
    :return: list of structure IDs (strings)
    """

    is_file = currPDB.endswith(".pdb") and os.path.isfile(currPDB)
    chain_list = list(chains)

    # Determine mode if "auto"
    if mode == "auto":
        try:
            total_length = getTotalProteinLength(currPDB, chains)
            LOGGER.info(f"Total sequence length of {currPDB} ({'+'.join(chains)}): {total_length} residues.")
            mode = "similar" if total_length < 550 else "diverse"
        except Exception as e:
            LOGGER.warn(f"Could not determine sequence length for {currPDB}. Defaulting to 'diverse'. {str(e)}")
            mode = "diverse"

    # Set Dali params
    if mode == "similar":
        length = 0.8 if length is None else length
        rmsd = 2.0 if rmsd is None else rmsd
        search_func = searchDal
    else:
        length = 0.90 if length is None else length
        rmsd = 1.0 if rmsd is None else rmsd
        search_func = searchDali

    LOGGER.info(f"Using '{mode}' mode with length={length}, rmsd={rmsd}, Z={Z}")

    # Monomer
    if len(chain_list) == 1:
        chain = chain_list[0]
        dali_rec = search_func(currPDB, chain)
        while not dali_rec.isSuccess:
            dali_rec.fetch()

        pdb_ids = dali_rec.filter(cutoff_len=length, cutoff_rmsd=rmsd, cutoff_Z=Z)
        if is_file:
            pdb_ids.insert(0, currPDB)
        savePDBList(pdb_ids, currPDB, chain)

        return pdb_ids

    # Multimer
    chain_results = {}
    for chain in chain_list:
        LOGGER.info(f"Processing {currPDB} chain {chain}")
        dali_rec = search_func(currPDB, chain)
        while not dali_rec.isSuccess:
            dali_rec.fetch()
        chain_results[chain] = set(dali_rec.filter(cutoff_len=length, cutoff_rmsd=rmsd, cutoff_Z=Z))

    merged_results = mergeChains(currPDB, chain_results, chain_list)
    if not merged_results:
        LOGGER.warn(f"No valid merged multi-chain homologs found for {currPDB}. Returning only reference.")
        return [currPDB] if is_file else []

    if not is_file:
        ref = currPDB + chains
        merged_results.insert(0, ref)
        merged_results = [item for item in merged_results if item != ref]
        merged_results.insert(0, ref)
    else:
        merged_results.insert(0, currPDB)
    
    saveMultiChainResults(merged_results, currPDB)
    return merged_results

In [28]:
currPDB = "1fxv"
chains = 'AB'  # Multi-chain query

# merged_results = getMergedMultiChainEnsemble(currPDB, chains, length=0.9, rmsd=1, Z=10)
merged_results = getEnsembleWithModes(currPDB, chains)


@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
@> 1fxv downloaded (1fxv.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> 6548 atoms and 1 coordinate set(s) were parsed in 0.10s.
@> Secondary structures were assigned to 459 residues.
@> Total sequence length of 1fxv (A+B): 764 residues.
@> Using 'diverse' mode with length=0.9, rmsd=1.0, Z=10
@> Processing 1fxv chain A
@> Submitted Dali search for PDB "1fxvA".
@> http://ekhidna2.biocenter.helsinki.fi/barcosel/tmp//1fxvA/
@> Dali results were fetched in 0.2s.   
@> Obtained 129 PDB chains from Dali for 1fxvA.
@> 111 PDBs have been filtered out from 129 Dali hits (remaining: 18).
@> Processing 1fxv chain B
@> Submitted Dali search for PDB "1fxvB".
@> http://ekhidna2.biocenter.helsinki.fi/barcosel/tmp//1fxvB/
@> Dali results were fetched in 0.2s.   
@> Obtained 3020 PDB chains from Dali for 1fxvB.
@> 3008 PDBs have been filtered out from 3020 Dali hits (remaining: 

In [29]:
merged_results

['1fxvAB',
 '6nvxAB',
 '6nvyCD',
 '3k3wAB',
 '8brqAB',
 '8brtAB',
 '3ml0AB',
 '8brrCD',
 '8brsAB',
 '6nvwAB']

# GNM motions

In [37]:
def motionFromEnsemble_AF(input_data, ref=None, from_file=False):
    """
    Computes GNM (Gaussian Network Model) from an ensemble list or a text file.

    Args:
        input_data (list[str] or str): List of PDB+Chain identifiers (e.g. '1tklA' or '1tklAB')
                                       or file path to such a list.
        ref (Atomic): Reference structure for alignment (optional).
        from_file (bool): If True, reads list from file.

    Returns:
        tuple: (GNM object, average eigenvalues, average eigenvectors)
    """
    import os

    if from_file:
        try:
            with open(input_data, "r") as f:
                lines = f.readlines()[1:]  # Skip header
                pdb_chains = [line.strip().replace("\t", "") for line in lines]
        except FileNotFoundError:
            LOGGER.error(f"File {input_data} not found.")
            return None, None, None
    else:
        pdb_chains = input_data

    if not pdb_chains:
        LOGGER.error("No PDB entries provided.")
        return None, None, None

    if len(pdb_chains) == 1:
        LOGGER.warning("Only one structure provided; duplicating it for ensemble.")
        pdb_chains.append(pdb_chains[0])

    LOGGER.info(f"Processing {len(pdb_chains)} PDB entries.")

    all_atoms = []

    for entry in pdb_chains:
        if os.path.isfile(entry):
            # It's a custom .pdb file (e.g., "CPOX_AF.pdb")
            ag = parsePDB(entry, subset="ca")
            all_atoms.append(ag)
        else:
            # It's a PDB ID with one or more chains (e.g., "1tklA", "1tklAB")
            pdb_id = entry[:4]
            chain_ids = entry[4:]

            if not chain_ids:
                ag = parsePDB(pdb_id, subset="ca")
                all_atoms.append(ag)
            else:
                for chain in chain_ids:
                    ag = parsePDB(pdb_id, chain=chain, subset="ca")
                    all_atoms.append(ag)

    if ref:
        ensemble = buildPDBEnsemble(all_atoms, ref=ref)
    else:
        ensemble = buildPDBEnsemble(all_atoms)

    gnms = calcEnsembleENMs(ensemble, model="GNM", trim="reduce", n_modes=None)

    eigVals = gnms.getEigvals()
    averageEigVals = sort(np.mean(eigVals, axis=0))
    eigVects = gnms.getEigvecs()
    averageEigVecs = np.mean(eigVects, axis=0)

    LOGGER.info(f"GNM computation completed with {len(eigVals)} eigenvalues.")

    return gnms, averageEigVals, averageEigVecs


def motionFromEnsemble_PDB(input_data, ref=None, from_file=False):
    """
    Computes GNM (Gaussian Network Model) from an ensemble list or a text file.

    :param input_data: Either a list of PDB chain identifiers or a path to a text file containing them.
    :type input_data: list of str or str (file path)

    :param ref: Reference structure for ensemble alignment. Default is the first structure in your list.
    :type ref: Atomic object (optional)

    :param from_file: Whether input_data is a file path. Default is False.
    :type from_file: bool

    :return: GNM object, average eigenvalues, average eigenvectors
    :rtype: tuple (GNM, np.array, np.array)
    """

    # Read data from file if from_file=True
    if from_file:
        try:
            with open(input_data, "r") as f:
                lines = f.readlines()[1:]  # Skip header
                pdb_chains = [line.strip().replace("\t", "") for line in lines]
        except FileNotFoundError:
            LOGGER.error(f"File {input_data} not found.")
            return None, None, None
    else:
        pdb_chains = input_data  # Directly use provided list

    LOGGER.info(f"Processing {len(pdb_chains)} PDB entries.")
    
    # Extract PDB IDs and Chains separately
    if len(pdb_chains[0]) == 5:
        ags = parsePDB(pdb_chains, subset="ca")
    else:
        mergeIDs = [entry[:4] for entry in pdb_chains]  # First 4 chars = PDB ID
        Chains = [entry[4:] for entry in pdb_chains]  # Remaining chars = Chains
        
        ags = parsePDB(mergeIDs, subset="ca", chain=Chains[0])

    # Build ensemble with or without ref
    if ref:
        dali_ens = buildPDBEnsemble(ags, ref=ref)
    else:
        dali_ens = buildPDBEnsemble(ags)

    # Compute GNM
    gnms = calcEnsembleENMs(dali_ens, model="GNM", trim="reduce", n_modes=None)

    # Extract eigenvalues & eigenvectors
    eigVals = gnms.getEigvals()
    averageEigVals = sort(np.mean(eigVals, axis=0))
    eigVects = gnms.getEigvecs()
    averageEigVecs = np.mean(eigVects, axis=0)

    LOGGER.info(f"GNM computation completed with {len(eigVals)} eigenvalues.")

    return gnms, averageEigVals, averageEigVecs


def motionFromEnsemble(input_data, ref=None, from_file=False):
    if '.pdb' in input_data[0]:
        gnms, avg_eigvals, avg_eigvecs = motionFromEnsemble_AF(input_data, ref=ref)
    else:
        gnms, avg_eigvals, avg_eigvecs = motionFromEnsemble_PDB(input_data, ref=ref)
    return gnms, avg_eigvals, avg_eigvecs

In [38]:
# gnms, avg_eigvals, avg_eigvecs = motionFromEnsemble("Results/4m11_MultiChainEnsemble.txt", from_file=True)
gnms, avg_eigvals, avg_eigvecs = motionFromEnsemble(merged_results)
# gnms, avg_eigvals, avg_eigvecs = motionFromEnsemble(ids)
# gnms_2, avg_eigvals_2, avg_eigvecs_2 = motionFromEnsemble("./Results/3d4s_A_Ensemble.txt", from_file=True, ref=calphas)

@> Processing 10 PDB entries.
@> 10 PDBs were parsed in 1.76s. 
@> Starting iterative superposition:              
@> Step #1: RMSD difference = 1.7097e+00
@> Step #2: RMSD difference = 9.9024e-03
@> Step #3: RMSD difference = 3.8453e-04
@> Step #4: RMSD difference = 1.7184e-05
@> Iterative superposition completed in 0.02s.
@> Final superposition to calculate transformations.
@> Superposition completed in 0.00 seconds.
@> Ensemble (10 conformations) were built in 12.57s.
@> WARNING last 67 modes for 695 modes from MaskedGNM 1fxvAB_ca reduced has been discarded because at least one conformation has only 695 modes
@> WARNING last 8 modes for 695 modes from MaskedGNM 6nvxAB_ca reduced has been discarded because at least one conformation has only 695 modes
@> WARNING last 8 modes for 695 modes from MaskedGNM 6nvyAB_ca reduced has been discarded because at least one conformation has only 695 modes
@> WARNING last 32 modes for 695 modes from MaskedGNM 3k3wAB_ca reduced has been discarded bec

# 33% contribution

In [39]:
currNumModes = findModesForThreshold(avg_eigvals, 0.33)
currNumModes

3

# get hinges

In [40]:
avg_eigvecs.shape

(763, 695)

In [41]:
avg_eigvecs[0]

array([-4.78378862e-02, -3.37788452e-02,  6.76811092e-04, -3.37763473e-02,
       -9.46079232e-02, -1.05071298e-01, -7.44134478e-03, -6.19239447e-02,
       -3.98478659e-02,  2.28652627e-02, -1.55970775e-01, -1.22421827e-01,
        2.79783765e-02, -1.22219585e-01, -1.59561026e-04,  5.12929889e-02,
        9.09434156e-02, -5.33292287e-02, -1.85394867e-02,  1.92959421e-02,
       -1.03438130e-01, -1.18935115e-01, -7.07847444e-02, -7.16886310e-02,
        4.57450430e-02,  5.79056747e-02, -9.31350581e-02, -4.06748783e-02,
       -1.16044853e-01,  1.02716120e-01,  4.61868274e-02,  5.25684515e-02,
       -1.25159044e-02,  2.32568808e-02,  6.04316092e-02, -1.91231195e-01,
       -7.06677076e-02,  6.20866149e-02,  5.61157466e-02, -5.19804935e-02,
        7.12165965e-02,  6.65603928e-02,  3.33874385e-02,  5.82396920e-01,
        2.32943651e-01,  1.85190834e-01,  8.14004904e-02,  1.69664083e-02,
       -1.96209153e-02, -8.35595006e-02, -7.39758084e-02, -1.58435991e-02,
       -1.93768997e-01,  

In [42]:
primary, minor = identifyHingeSites(avg_eigvecs, avg_eigvals)

print("Primary Hinges:", primary, len(primary))
print("Minor Hinges:", minor)

@> Auto-selected 3 modes based on contribution threshold.


Primary Hinges: [0, 2, 21, 22, 23, 24, 45, 46, 75, 76, 77, 78, 135, 136, 138, 139, 140, 141, 142, 143, 146, 147, 184, 185, 187, 188, 190, 191, 192, 193, 194, 195, 205, 206, 207, 229, 230, 236, 237, 238, 239, 254, 255, 257, 258, 273, 274, 282, 283, 344, 345, 352, 353, 354, 355, 375, 377, 383, 384, 396, 397, 407, 408, 413, 414, 415, 416, 418, 419, 432, 433, 442, 443, 446, 447, 466, 467, 476, 477, 478, 479, 488, 489, 496, 497, 498, 500, 503, 504, 559, 560, 561, 562, 563, 579, 580, 581, 582, 590, 591, 593, 594, 621, 622, 623, 624, 657, 658, 677, 678, 682, 683, 684, 685, 708, 709, 731, 732, 755, 756] 120
Minor Hinges: [1, 25, 137, 139, 186, 189, 194, 205, 259, 356, 376, 406, 465, 475, 495, 499, 501, 502]


In [50]:
# hinge_residues = mapHingesToResidues(calphas, minor, reference_selection=calphas)

hinge_residues = {'primary': mapHingesToResidues(currPDB, primary, chain=chains), 'minor': mapHingesToResidues(currPDB, minor, chain=chains)}

print("Hinge Residues:", hinge_residues)
len(hinge_residues)


@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
@> 1fxv downloaded (1fxv.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> 763 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> Secondary structures were assigned to 459 residues.
@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
@> 1fxv downloaded (1fxv.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> 763 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> Secondary structures were assigned to 459 residues.


Hinge Residues: {'primary': ['SER3(A)', 'SER5(A)', 'THR24(A)', 'TRP25(A)', 'HIS26(A)', 'LEU27(A)', 'ARG48(A)', 'SER49(A)', 'ILE78(A)', 'ARG79(A)', 'ALA80(A)', 'GLN81(A)', 'PHE138(A)', 'VAL139(A)', 'THR141(A)', 'MET142(A)', 'ALA143(A)', 'ASN144(A)', 'ARG145(A)', 'PHE146(A)', 'SER149(A)', 'THR150(A)', 'PRO187(A)', 'THR188(A)', 'ILE190(A)', 'ALA191(A)', 'GLN193(A)', 'GLU194(A)', 'SER195(A)', 'ASN196(A)', 'TYR197(A)', 'PRO198(A)', 'THR208(A)', 'SER1(B)', 'ASN2(B)', 'PHE24(B)', 'GLY25(B)', 'TYR31(B)', 'THR32(B)', 'TYR33(B)', 'GLY34(B)', 'PRO49(B)', 'PHE50(B)', 'TYR52(B)', 'PRO53(B)', 'THR68(B)', 'ALA69(B)', 'ILE77(B)', 'PHE78(B)', 'LYS139(B)', 'SER140(B)', 'GLU147(B)', 'LEU148(B)', 'ALA149(B)', 'SER150(B)', 'ALA170(B)', 'LYS172(B)', 'ASN178(B)', 'TRP179(B)', 'VAL191(B)', 'HIS192(B)', 'GLY202(B)', 'HIS203(B)', 'PRO208(B)', 'VAL209(B)', 'PRO210(B)', 'GLY211(B)', 'GLY213(B)', 'LYS214(B)', 'PRO227(B)', 'LYS228(B)', 'ILE237(B)', 'ALA238(B)', 'ALA241(B)', 'ASN242(B)', 'ALA261(B)', 'ASP262(B)', 'L

2

In [44]:
# Run hinge analysis and save results
saveHingeResidues("1fxv", "AB", avg_eigvecs, avg_eigvals)

@> Auto-selected 3 modes based on contribution threshold.
@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
@> 1fxv downloaded (1fxv.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> 763 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> Secondary structures were assigned to 459 residues.
@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
@> 1fxv downloaded (1fxv.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> 763 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> Secondary structures were assigned to 459 residues.
@> Hinge residues saved to Results/1fxv_AB_HingeResidues.txt


# Find Drug binding sites

In [54]:
def parse_pdb_gz(pdb_id, pdb_dir="./"):
    """
    Parses a `.pdb.gz` file and extracts protein and ligand atomic coordinates.

    :param pdb_id: The PDB ID of the structure.
    :type pdb_id: str

    :param pdb_dir: Directory containing the `.pdb.gz` files.
    :type pdb_dir: str

    :return: Dictionary with protein and ligand atom coordinates.
    :rtype: dict
    """
    pdb_path = os.path.join(pdb_dir, f"{pdb_id}.pdb.gz")

    if not os.path.exists(pdb_path):
        LOGGER.warning(f"PDB file {pdb_path} not found.")
        return None

    protein_atoms = {}  # {chain: [(residue_name, residue_id, atom_name, x, y, z), ...]}
    ligand_atoms = {}   # {ligand_id: [(chain, residue_id, atom_name, x, y, z), ...]}

    # Open and read .pdb.gz file
    with gzip.open(pdb_path, "rt") as f:
        for line in f:
            if line.startswith("ATOM"):  # Protein atoms
                atom_name = line[12:16].strip()
                residue_name = line[17:20].strip()
                chain = line[21].strip()
                residue_id = line[22:26].strip()
                x, y, z = map(float, [line[30:38], line[38:46], line[46:54]])

                if chain not in protein_atoms:
                    protein_atoms[chain] = []
                protein_atoms[chain].append((residue_name, residue_id, atom_name, x, y, z))

            elif line.startswith("HETATM"):  # Ligand atoms (excluding water)
                residue_name = line[17:20].strip()
                if residue_name == "HOH":
                    continue  # Skip water molecules

                atom_name = line[12:16].strip()
                chain = line[21].strip()
                residue_id = line[22:26].strip()
                x, y, z = map(float, [line[30:38], line[38:46], line[46:54]])

                ligand_id = f"{residue_name}_{residue_id}"
                if ligand_id not in ligand_atoms:
                    ligand_atoms[ligand_id] = []
                ligand_atoms[ligand_id].append((chain, residue_id, atom_name, x, y, z))

    LOGGER.info(f"Parsed {pdb_id}.pdb.gz: Found {len(protein_atoms)} protein chains and {len(ligand_atoms)} ligands.")
    return {"protein": protein_atoms, "ligand": ligand_atoms}

def parse_pdb_AF(pdb_path):
    """
    Parses a `.pdb` or `.txt` file and extracts protein and ligand atomic coordinates.

    :param pdb_path: Path to the PDB file (.pdb or .txt).
    :type pdb_path: str

    :return: Dictionary with protein and ligand atom coordinates.
    :rtype: dict
    """
    if not os.path.exists(pdb_path):
        LOGGER.warning(f"PDB file {pdb_path} not found.")
        return None

    protein_atoms = {}  # {chain: [(residue_name, residue_id, atom_name, x, y, z), ...]}
    ligand_atoms = {}   # {ligand_id: [(chain, residue_id, atom_name, x, y, z), ...]}

    with open(pdb_path, "r") as f:
        for line in f:
            if line.startswith("ATOM"):
                atom_name = line[12:16].strip()
                residue_name = line[17:20].strip()
                chain = line[21].strip()
                residue_id = line[22:26].strip()
                x, y, z = map(float, [line[30:38], line[38:46], line[46:54]])

                protein_atoms.setdefault(chain, []).append(
                    (residue_name, residue_id, atom_name, x, y, z)
                )

            elif line.startswith("HETATM"):
                residue_name = line[17:20].strip()
                if residue_name == "HOH":
                    continue  # Skip water

                atom_name = line[12:16].strip()
                chain = line[21].strip()
                residue_id = line[22:26].strip()
                x, y, z = map(float, [line[30:38], line[38:46], line[46:54]])

                ligand_id = f"{residue_name}_{residue_id}"
                ligand_atoms.setdefault(ligand_id, []).append(
                    (chain, residue_id, atom_name, x, y, z)
                )

    LOGGER.info(f"Parsed {os.path.basename(pdb_path)}: Found {len(protein_atoms)} protein chains and {len(ligand_atoms)} ligands.")
    return {"protein": protein_atoms, "ligand": ligand_atoms}



def process_pdb_ligand_binding(pdb_list, threshold=5.0, pdb_dir="./", output_dir="Results"):
    """
    Identifies ligand binding sites for multiple PDB structures from `.pdb.gz` files.

    :param pdb_list: List of PDB structure identifiers.
    :type pdb_list: list

    :param threshold: Distance cutoff for ligand binding (default = 5.0 Å).
    :type threshold: float

    :param pdb_dir: Directory containing the `.pdb.gz` files.
    :type pdb_dir: str

    :param output_dir: Directory to save results.
    :type output_dir: str

    :return: Dictionary mapping PDB structures to their ligand binding sites.
    :rtype: dict
    """
    # Ensure Results folder exists
    os.makedirs(output_dir, exist_ok=True)

    binding_results = {}

    for pdb_entry in pdb_list:
        pdb_id = pdb_entry[:4]  # Try first 4 letters as PDB ID
        tried_paths = []
    
        # Try exact name (e.g., "CPOX_AF.pdb")
        pdb_path_1 = os.path.join(pdb_dir, f"{pdb_entry}")
    
        # Try gzipped PDB format (e.g., "1tkl.pdb.gz")
        pdb_path_2 = os.path.join(pdb_dir, f"{pdb_id}.pdb.gz")
        
        # Try them in order
        if os.path.exists(pdb_path_1):
            structure_file = pdb_path_1
            parsed_data = parse_pdb_AF(structure_file)
        elif os.path.exists(pdb_path_2):
            structure_file = pdb_id
            parsed_data = parse_pdb_gz(structure_file)
        else:
            structure_file = None
        
        if structure_file is None:
            LOGGER.warn(f"Skipping {pdb_entry}: no .pdb or .pdb.gz file found in {pdb_dir}")
            continue

        # Find ligand binding sites
        protein_atoms = parsed_data["protein"]
        ligand_atoms = parsed_data["ligand"]
        binding_sites = find_binding_residues(protein_atoms, ligand_atoms, threshold)

        if binding_sites:
            binding_results[pdb_id] = binding_sites

    # Save results to a TSV file
    output_file = os.path.join(output_dir, "LigandBindingSites.tsv")
    with open(output_file, "w") as f:
        f.write("PDB\tLigand\tChain\tBinding Residues\n")
        for pdb_id, ligands in binding_results.items():
            for ligand, chains in ligands.items():
                for chain, residues in chains.items():
                    f.write(f"{pdb_id}\t{ligand}\t{chain}\t{', '.join(residues)}\n")

    LOGGER.info(f"Ligand binding site results saved to {output_file}")
    return binding_results

In [55]:
binding_sites = process_pdb_ligand_binding(merged_results, threshold=5.0)

binding_sites

@> Parsed 1fxv.pdb.gz: Found 2 protein chains and 2 ligands.
@> Parsed 6nvx.pdb.gz: Found 2 protein chains and 10 ligands.
@> Parsed 6nvy.pdb.gz: Found 4 protein chains and 7 ligands.
@> Parsed 3k3w.pdb.gz: Found 2 protein chains and 1 ligands.
@> Parsed 8brq.pdb.gz: Found 2 protein chains and 6 ligands.
@> Parsed 8brt.pdb.gz: Found 2 protein chains and 5 ligands.
@> Parsed 3ml0.pdb.gz: Found 2 protein chains and 1 ligands.
@> Parsed 8brr.pdb.gz: Found 4 protein chains and 4 ligands.
@> Parsed 8brs.pdb.gz: Found 2 protein chains and 6 ligands.
@> Parsed 6nvw.pdb.gz: Found 2 protein chains and 2 ligands.
@> Ligand binding site results saved to Results/LigandBindingSites.tsv


{'1fxv': {'PNN_1001': {'B': ['ALA69',
    'GLN23',
    'ILE177',
    'PHE24',
    'PHE71',
    'PRO22',
    'SER1',
    'SER67',
    'THR68'],
   'A': ['ARG145', 'MET142', 'PHE146', 'SER149']},
  'CA_2001': {'A': ['GLU152', 'THR150'],
   'B': ['ARG199', 'ARG206', 'ASP252', 'ASP73', 'ASP76', 'PRO205', 'VAL75']}},
 '6nvx': {'EPE_301': {'A': ['ASP199', 'GLN197', 'ILE200', 'LYS201', 'THR198'],
   'B': ['ALA228', 'ALA233', 'GLN232', 'LYS229', 'SER226']},
  'GOL_302': {'A': ['ALA188',
    'GLN193',
    'GLU187',
    'ILE192',
    'LEU186',
    'PRO189',
    'PRO194',
    'THR190'],
   'B': ['ILE271', 'LYS250', 'SER249']},
  'GOL_303': {'A': ['ASP115',
    'GLY126',
    'HIS123',
    'LEU118',
    'LEU128',
    'PHE122',
    'PHE127',
    'PRO114']},
  'GOL_304': {'A': ['ASN151', 'ASP150', 'MET149', 'PHE148', 'TYR147'],
   'B': ['ALA69', 'ARG381', 'TYR257', 'TYR71']},
  'EPE_601': {'B': ['ASN166', 'GLU169', 'LYS168', 'VAL167']},
  'GOL_602': {'B': ['ARG328',
    'ASN331',
    'ASN332',
    'L

In [56]:
# Load FDA-approved ligands
approved_ligands = load_approved_ligands('FDA_Approved_Ligands_Filtered.txt')
# Filter binding sites
filtered_binding_sites = filter_ligands(binding_sites, approved_ligands)

filtered_binding_sites

{'8brs': {'BU3_602': {'A': ['MET145', 'MET149', 'PHE148'],
   'B': ['ALA67',
    'ALA69',
    'ASN182',
    'ASN245',
    'GLN23',
    'GLY21',
    'LEU181',
    'MET56',
    'PHE49',
    'PHE57',
    'PRO22',
    'SER1',
    'THR68',
    'TYR158',
    'VAL24']}}}

In [57]:
# Load FDA-approved ligands
approved_ligands = load_approved_ligands('FDA_Approved_Ligands_Filtered.txt')
# Filter binding sites
filtered_binding_sites = filter_ligands(binding_sites, approved_ligands)

filtered_binding_sites

{'8brs': {'BU3_602': {'A': ['MET145', 'MET149', 'PHE148'],
   'B': ['ALA67',
    'ALA69',
    'ASN182',
    'ASN245',
    'GLN23',
    'GLY21',
    'LEU181',
    'MET56',
    'PHE49',
    'PHE57',
    'PRO22',
    'SER1',
    'THR68',
    'TYR158',
    'VAL24']}}}

In [58]:
merged_results

['1fxvAB',
 '6nvxAB',
 '6nvyCD',
 '3k3wAB',
 '8brqAB',
 '8brtAB',
 '3ml0AB',
 '8brrCD',
 '8brsAB',
 '6nvwAB']

In [59]:
# Step 1: Extract sequences
sequences = extract_sequences(merged_results)
print (sequences)


@> 6548 atoms and 1 coordinate set(s) were parsed in 0.09s.
@> Secondary structures were assigned to 459 residues.
@> 12843 atoms and 1 coordinate set(s) were parsed in 0.18s.
@> Secondary structures were assigned to 489 residues.
@> 23638 atoms and 1 coordinate set(s) were parsed in 0.34s.
@> Secondary structures were assigned to 967 residues.
@> 6011 atoms and 1 coordinate set(s) were parsed in 0.08s.
@> Secondary structures were assigned to 399 residues.
@> 12411 atoms and 1 coordinate set(s) were parsed in 0.18s.
@> Secondary structures were assigned to 479 residues.
@> 12333 atoms and 1 coordinate set(s) were parsed in 0.18s.
@> Secondary structures were assigned to 476 residues.
@> 5996 atoms and 1 coordinate set(s) were parsed in 0.07s.
@> Secondary structures were assigned to 420 residues.
@> 23997 atoms and 1 coordinate set(s) were parsed in 0.34s.
@> Secondary structures were assigned to 946 residues.
@> 12322 atoms and 1 coordinate set(s) were parsed in 0.17s.
@> Secondary s

{'1fxv': {'A': ('SSSEIKIVRDEYGMPHIYANDTWHLFYGYGYVVAQDRLFQMEMARRSTQGTVAEVLGKDFVKFDKDIRRNYWPDAIRAQIAALSPEDMSILQGYADGMNAWIDKVNTNPETLLPKQFNTFGFTPKRWEPFDVAMIFVGTMANRFSDSTSEIDNLALLTALKDKYGVSQGMAVFNQLKWLVNPSAPTTIAVQESNYPLKFNQQNSQT', array([  3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 14

In [60]:
# Run best chain match finder
reference_pdb = list(sequences.keys())[0]  # First PDB as reference
best_chain_matches = find_best_chain_match(reference_pdb, sequences)
print("Best chain matches:", best_chain_matches)

@> Starting chain matching for 10 PDB structures against reference 1fxv...
@> Processing PDB 1/10: 1fxv
@>  Alignment complete: 1fxv → B (Target) ↔ B (Reference) | Score: 3007.0
@> Processing PDB 2/10: 6nvx
@>  Alignment complete: 6nvx → B (Target) ↔ B (Reference) | Score: 772.0
@> Processing PDB 3/10: 6nvy
@>  Alignment complete: 6nvy → D (Target) ↔ B (Reference) | Score: 790.5
@> Processing PDB 4/10: 3k3w
@>  Alignment complete: 3k3w → B (Target) ↔ B (Reference) | Score: 1145.0
@> Processing PDB 5/10: 8brq
@>  Alignment complete: 8brq → B (Target) ↔ B (Reference) | Score: 779.0
@> Processing PDB 6/10: 8brt
@>  Alignment complete: 8brt → B (Target) ↔ B (Reference) | Score: 774.0
@> Processing PDB 7/10: 3ml0
@>  Alignment complete: 3ml0 → B (Target) ↔ B (Reference) | Score: 1145.0
@> Processing PDB 8/10: 8brr
@>  Alignment complete: 8brr → D (Target) ↔ B (Reference) | Score: 766.5
@> Processing PDB 9/10: 8brs
@>  Alignment complete: 8brs → B (Target) ↔ B (Reference) | Score: 766.5
@> P

Best chain matches: {'1fxv': {'ref_chain': 'B', 'target_chain': 'B', 'alignment': ('SNMWVIGKSKAQDAKAIMVNGPQFGWYAPAYTYGIGLHGAGYDVTGNTPFAYPGLVFGHNGVISWGSTAGFGDDVDIFAERLSAEKPGYYLHNGKWVKMLSREETITVKNGQAETFTVWRTVHGNILQTDQTTQTAYAKSRAWDGKELASLLAWTHQMKAKNWQEWTQQAAKQALTINWYYADVNGNIGYVHTGAYPDRQSGHDPRLPVPGTGKWDWKGLLPFEMNPKVYNPQSGYIANWANSPQKDYPASDLFAFLWGGADRVTEIDRLLEQKPRLTADQAWDVIRQTSRQDLNLRLFLPTLQAATSGLTQSDPRRQLVETLTRWDGINLLNDDGKTWQQPGSAILNVWLTSMLKRTVVAAVPMPFDKWYSASGYETTQDGPTGSLNISVGAKILYEAVQGDKSPIPQAVDLFAGKPQQEVVLAALEDTWETLSKRYGNNVSNWKTPAMALTFRANNFFGVPQAAAEETRHQAEYQNRGTENDMIVFSPTTSDRPVLAWDVVAPGQSGFIAPDGTVDKHYEDQLKMYENFGRKSLWLTKQDVEAHKESQEVLHVQR', 'SNMWVIGKSKAQDAKAIMVNGPQFGWYAPAYTYGIGLHGAGYDVTGNTPFAYPGLVFGHNGVISWGSTAGFGDDVDIFAERLSAEKPGYYLHNGKWVKMLSREETITVKNGQAETFTVWRTVHGNILQTDQTTQTAYAKSRAWDGKELASLLAWTHQMKAKNWQEWTQQAAKQALTINWYYADVNGNIGYVHTGAYPDRQSGHDPRLPVPGTGKWDWKGLLPFEMNPKVYNPQSGYIANWANSPQKDYPASDLFAFLWGGADRVTEIDRLLEQKPRLTADQAWDVIRQTSRQDLNLRLFLPTLQAATSGLTQSDPRRQLVETLTRWDGINLLNDDGKTWQQPGSAILNVWLTSML

In [61]:
filtered_binding_sites


{'8brs': {'BU3_602': {'A': ['MET145', 'MET149', 'PHE148'],
   'B': ['ALA67',
    'ALA69',
    'ASN182',
    'ASN245',
    'GLN23',
    'GLY21',
    'LEU181',
    'MET56',
    'PHE49',
    'PHE57',
    'PRO22',
    'SER1',
    'THR68',
    'TYR158',
    'VAL24']}}}

In [62]:
# Run mapping function
mapped_binding_sites = map_binding_sites_to_reference(best_chain_matches, sequences, filtered_binding_sites)
print("Mapped Binding Sites to Reference:", mapped_binding_sites)


@> Reference PDB: 1fxv
@> Chains in Reference: AB
@> Available Binding Sites: 8brs


Mapped Binding Sites to Reference: {'A': {}, 'B': {'S67': 1.0, 'A69': 4.0, 'A182': 4.0, 'Q245': -1.0, 'Q23': -2.0, 'G21': 6.0, 'Y181': -1.0, 'V56': 1.0, 'P49': 7.0, 'F57': -4.0, 'P22': 7.0, 'S1': 4.0, 'T68': 5.0, 'M158': -1.0, 'F24': -1.0}}


In [63]:

# Identify similar chains in reference structure
similar_chains = find_similar_chains(reference_pdb, sequences)
print("Similar Chains in Reference Structure:", similar_chains)



Similar Chains in Reference Structure: []


In [64]:

# Run chain-to-chain mapping
updated_binding_sites = propagate_binding_sites(mapped_binding_sites, similar_chains)
print("Final Mapped Binding Sites with Chain-to-Chain Mapping:", updated_binding_sites)


Final Mapped Binding Sites with Chain-to-Chain Mapping: {'A': {}, 'B': {'S67': 1.0, 'A69': 4.0, 'A182': 4.0, 'Q245': -1.0, 'Q23': -2.0, 'G21': 6.0, 'Y181': -1.0, 'V56': 1.0, 'P49': 7.0, 'F57': -4.0, 'P22': 7.0, 'S1': 4.0, 'T68': 5.0, 'M158': -1.0, 'F24': -1.0}}


In [65]:
set(mapped_binding_sites['A']) == set(mapped_binding_sites['B'])

False

In [66]:
converted_binding_sites = convert_binding_sites_to_three_letter(updated_binding_sites)

print("Converted Binding Sites:", converted_binding_sites)

Converted Binding Sites: {'A': {}, 'B': {'SER67': 1.0, 'ALA69': 4.0, 'ALA182': 4.0, 'GLN245': -1.0, 'GLN23': -2.0, 'GLY21': 6.0, 'TYR181': -1.0, 'VAL56': 1.0, 'PRO49': 7.0, 'PHE57': -4.0, 'PRO22': 7.0, 'SER1': 4.0, 'THR68': 5.0, 'MET158': -1.0, 'PHE24': -1.0}}


# p values

In [67]:
print (hinge_residues)

{'primary': ['SER3(A)', 'SER5(A)', 'THR24(A)', 'TRP25(A)', 'HIS26(A)', 'LEU27(A)', 'ARG48(A)', 'SER49(A)', 'ILE78(A)', 'ARG79(A)', 'ALA80(A)', 'GLN81(A)', 'PHE138(A)', 'VAL139(A)', 'THR141(A)', 'MET142(A)', 'ALA143(A)', 'ASN144(A)', 'ARG145(A)', 'PHE146(A)', 'SER149(A)', 'THR150(A)', 'PRO187(A)', 'THR188(A)', 'ILE190(A)', 'ALA191(A)', 'GLN193(A)', 'GLU194(A)', 'SER195(A)', 'ASN196(A)', 'TYR197(A)', 'PRO198(A)', 'THR208(A)', 'SER1(B)', 'ASN2(B)', 'PHE24(B)', 'GLY25(B)', 'TYR31(B)', 'THR32(B)', 'TYR33(B)', 'GLY34(B)', 'PRO49(B)', 'PHE50(B)', 'TYR52(B)', 'PRO53(B)', 'THR68(B)', 'ALA69(B)', 'ILE77(B)', 'PHE78(B)', 'LYS139(B)', 'SER140(B)', 'GLU147(B)', 'LEU148(B)', 'ALA149(B)', 'SER150(B)', 'ALA170(B)', 'LYS172(B)', 'ASN178(B)', 'TRP179(B)', 'VAL191(B)', 'HIS192(B)', 'GLY202(B)', 'HIS203(B)', 'PRO208(B)', 'VAL209(B)', 'PRO210(B)', 'GLY211(B)', 'GLY213(B)', 'LYS214(B)', 'PRO227(B)', 'LYS228(B)', 'ILE237(B)', 'ALA238(B)', 'ALA241(B)', 'ASN242(B)', 'ALA261(B)', 'ASP262(B)', 'LEU271(B)', 'GLU2

In [68]:
print (converted_binding_sites)

{'A': {}, 'B': {'SER67': 1.0, 'ALA69': 4.0, 'ALA182': 4.0, 'GLN245': -1.0, 'GLN23': -2.0, 'GLY21': 6.0, 'TYR181': -1.0, 'VAL56': 1.0, 'PRO49': 7.0, 'PHE57': -4.0, 'PRO22': 7.0, 'SER1': 4.0, 'THR68': 5.0, 'MET158': -1.0, 'PHE24': -1.0}}


In [69]:
sequences

{'1fxv': {'A': ('SSSEIKIVRDEYGMPHIYANDTWHLFYGYGYVVAQDRLFQMEMARRSTQGTVAEVLGKDFVKFDKDIRRNYWPDAIRAQIAALSPEDMSILQGYADGMNAWIDKVNTNPETLLPKQFNTFGFTPKRWEPFDVAMIFVGTMANRFSDSTSEIDNLALLTALKDKYGVSQGMAVFNQLKWLVNPSAPTTIAVQESNYPLKFNQQNSQT',
   array([  3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
           16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
           29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
           42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
           55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
           68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
           81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
           94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
          107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
          120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
          133, 134, 135,

In [70]:
process_sites(hinge_residues, converted_binding_sites, sequences, trimmed=False)

{'hinge_sites': {'primary': {'A': {('ALA', 80),
    ('ALA', 143),
    ('ALA', 191),
    ('ARG', 48),
    ('ARG', 79),
    ('ARG', 145),
    ('ASN', 144),
    ('ASN', 196),
    ('GLN', 81),
    ('GLU', 194),
    ('HIS', 26),
    ('ILE', 78),
    ('ILE', 190),
    ('LEU', 27),
    ('MET', 142),
    ('PHE', 138),
    ('PRO', 187),
    ('SER', 3),
    ('SER', 5),
    ('SER', 49),
    ('SER', 149),
    ('SER', 195),
    ('THR', 24),
    ('THR', 150),
    ('THR', 188),
    ('THR', 208),
    ('TRP', 25),
    ('TYR', 197),
    ('VAL', 139)},
   'B': {('ALA', 69),
    ('ALA', 149),
    ('ALA', 170),
    ('ALA', 238),
    ('ALA', 241),
    ('ALA', 261),
    ('ALA', 504),
    ('ARG', 291),
    ('ARG', 357),
    ('ARG', 479),
    ('ASN', 2),
    ('ASN', 178),
    ('ASN', 242),
    ('ASN', 295),
    ('ASN', 388),
    ('ASN', 478),
    ('ASP', 262),
    ('ASP', 284),
    ('ASP', 293),
    ('GLN', 273),
    ('GLN', 292),
    ('GLN', 419),
    ('GLN', 473),
    ('GLN', 477),
    ('GLN', 550),
    ('GL

In [71]:
# Compute p-value using processed data
p_value_result = compute_p_value(process_sites(hinge_residues, converted_binding_sites, sequences, trimmed=False), sequences)

# Display result
p_value_result

@> Results written to: Results/1fxv_Drug_vs_Hinges.txt


{'p_value': 0.009224562417850968, 'N': 763, 'M': 10, 'n': 114, 'k': 5}

In [72]:
# Compute p-value using processed data
p_value_result = compute_p_value(process_sites(hinge_residues, converted_binding_sites, sequences, trimmed=True), sequences)

# Display result
p_value_result

@> Results written to: Results/1fxv_Drug_vs_Hinges.txt


{'p_value': 0.006972575248320512, 'N': 763, 'M': 10, 'n': 107, 'k': 5}